In [1]:
from algorithms import *
from snake_environment import *
from states_bracket import *

In [2]:
def print_q_value(dictionary):
    for d in dictionary:
        print(f"State ({d[0]}, {d[1]}), Action {d[2]} : Value {dictionary[d]}")

In [ ]:
# Bracketer
bracketer = FoodRelativePositionBraket()
# General Settings 
gamma = 0.9
lr_v = 0.15
epsilon = 0.4
n_episodes = 10000

Proviamo il QLearning

In [ ]:
# Environment
env = SnakeEnv(render_mode="nonhuman")
# Bracketer
bracketer = FoodRelativePositionBraket()
# Sarsa
gamma = 0.9
lr_v = 0.15
Q_p = QLearning(env.action_space.n, gamma=gamma, lr_v=lr_v)
epsilon = 0.4

n_episodes = 5000

performance_traj_SARSA = np.zeros(n_episodes)

state, _ = env.reset()

for i in range(n_episodes):

    done = False
    keep = True

    env.reset()
    state = bracketer.bracket(env._get_obs())
    action = Q_p.get_action_epsilon_greedy(state, eps = epsilon)

    while not done and keep:

        new_s, reward, done, trunc, inf = env.step(action)
        if inf != {}:
            action = inf["act"]
        new_s = bracketer.bracket(new_s)
        
        # Keeps track of performance for each episode
        performance_traj_SARSA[i] += reward
        
        new_a = Q_p.get_action_epsilon_greedy(new_s, epsilon)

        Q_p.single_step_update(state, action, reward, new_s, new_a, done)
        
        action = new_a
        state = new_s

        keep = env.render()
    if i % 500 == 0:
        print(i)

env.close()

0
500
1000
1500
2000
2500
3000
3500


In [ ]:
path = "./models/"
Q_p.save(f"{path}gamma {gamma} lr {lr_v} epsilon {epsilon} episodes {n_episodes}")

In [5]:
done = False
keep = True
env = SnakeEnv(render_mode="human")

while not done and keep:
    action = Q_p.get_action_greedy(state)
    state, reward, done, trunc, inf = env.step(action)
    state = bracketer.bracket(state)
    keep = env.render()

env.close()

Proviamo SARSA

In [ ]:
# Environment
env = SnakeEnv(render_mode="nonhuman")

SARSA_p = SARSA(env.action_space.n, gamma=gamma, lr_v=lr_v)
SARSA_p.learning(env, epsilon, n_episodes, bracketer)

0


In [ ]:
path = "./models/"
SARSA_p.save(f"{path}gamma {gamma} lr {lr_v} epsilon {epsilon} episodes {n_episodes}")

In [6]:
SARSA_p.upload(f"{path}gamma {gamma} lr {lr_v}")

In [8]:
done = False
keep = True
env = SnakeEnv(render_mode="human")

while not done and keep:
    action = SARSA_p.get_action_greedy(state)
    state, reward, done, trunc, inf = env.step(action)
    state = bracketer.bracket(state)
    keep = env.render()

env.close()

Setting gamma to 0.999 (so having a time horizon of 1000 and a maximum number of steps of 1000) make the agent learn something about how to approach food in order to prevent to end in its own tail. 